Hey there!

Today I'd like to talk about **\__slots__**.

The inspiration for this article comes from a blog post about Python data structures published by [Dan Bader](https://dbader.org/blog/records-structs-and-data-transfer-objects-in-python) and the small iteration we then had on this [gist](https://gist.github.com/barrachri/c7922df84eb171eaa45e466b1b790bce) to check their performances.

For all the examples you are going to see I am using __Python 3.6.2__.

So what are slots? **\__slots__** are a different way to define the attributes storage for classes in Python.

If this is not clear bear with me.

In [1]:
# we use getsizeof to get the size of our objects
from sys import getsizeof
from sys import version as python_version
print(python_version)

3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:14:59) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]


In [2]:
class PythonClass():
    """This is a simple Python class"""
    
    def __init__(self, message):
        """Init method, nothing special here"""
        self.message = message
        self.capital_message = self.make_it_bigger()
    
    def make_it_bigger(self):
        """Do something with your attributes"""
        return self.message.upper()
    
    def scream_message(self):
        """Print the capital_message attribute of the instance"""
        print(self.capital_message) 
        
my_instance = PythonClass("my message")

So we have a class, `PythonClass`, and 1 instance of this class, `my_instance`.

Where are `message` and `capital_message` stored?

__Python__ uses a special attribute call __dict__ to store the instance's attributes:

In [3]:
[element for element in dir(my_instance) if element == '__dict__']

['__dict__']

In [4]:
my_instance.__dict__

{'capital_message': 'MY MESSAGE', 'message': 'my message'}

In [5]:
my_instance.new_message = "This is a new message"

In [6]:
my_instance.__dict__

{'capital_message': 'MY MESSAGE',
 'message': 'my message',
 'new_message': 'This is a new message'}

In [7]:
my_instance.__dict__['another_new_message'] = "Yet a new message"

In [8]:
my_instance.__dict__

{'another_new_message': 'Yet a new message',
 'capital_message': 'MY MESSAGE',
 'message': 'my message',
 'new_message': 'This is a new message'}

As you can see I can add new attributes to `my_instance` using both `my_instance.name_of_the_attribute` or `my_instance.__dict__['name_of_the_attribute']` notation.

So for normal classes Python uses a __dict__ to store instance's attributes.

### Is this bad or good?

Well, neither bad or good, [__dicts__ are awesome](https://www.youtube.com/watch?v=npw4s1QTmPg) but not perfect, because there is always a trade-off.

With a dict you have a constant lookup time, so the access time is more or less O(1) (it doesn't depend from the size of the dictionary), but because it's a mutable object and it can grow, it's a lot heavier (it has to allocate space for this).

Let's look at the **\__slots__** now.

In [9]:
class PythonClassWithSlots():
    """This is a simple Python class"""
    
    __slots__ = ["message", "capital_message"]
    
    def __init__(self, message):
        """Init method, nothing special here"""
        self.message = message
        self.capital_message = self.make_it_bigger() 
        
    def make_it_bigger(self):
        """Print the message attribute of the instance"""
        return self.message.upper()
    
    def scream_message(self):
        """Print the message attribute of the instance"""
        print(self.capital_message) 
        
my_instance = PythonClassWithSlots("my message")

In [10]:
[element for element in dir(my_instance) if element == '__dict__']

[]

So we don't have an attribute call `__dict__` inside our instance.

But we have a new attribute called **\__slots__**.

In [11]:
[element for element in dir(my_instance) if element == '__slots__']

['__slots__']

Can we access our attributes as we do with normal classes?

Indeed.

In [12]:
my_instance.message

'my message'

In [13]:
my_instance.capital_message

'MY MESSAGE'

Can we add new attributes?

In [14]:
my_instance.new_message = "This is a new message"

AttributeError: 'PythonClassWithSlots' object has no attribute 'new_message'

__No, we can't__.

But we can use the attributes that we defined during the class declaration, inside **\__slots__**.

In [15]:
my_instance.message = "Just putting something new here"

### But why would you need to use **slots** when you have a dict?
Well the answer is that \__slots__ are a *lot lighter and slightly faster*.

Are _slots_ based classes lighter than normal classes?

The answer should be yes, **but getting the size of an object is not that easy**.

In [16]:
my_instance_without_slots = PythonClass("my message")
my_instance_with_slots = PythonClassWithSlots("my message")

In [17]:
getsizeof(my_instance_without_slots)

56

In [18]:
getsizeof(my_instance_with_slots)

56

### mmm.....normal classes should be heavier, how is that possible?

With [**getsizeof**](https://docs.python.org/3/library/sys.html#sys.getsizeof) we get the size in **bytes** of our object but not of all the others referenced objects.
So in our case it should be in this way:

In [19]:
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(112, 56)

Now it makes a lot more sense.

In [20]:
my_instance_without_slots.new_attribute_1 = "This is a new attribute"
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(240, 56)

As you can see the size of **\__dict__** changes as long as we add new elements.

In [21]:
len(my_instance_without_slots.__dict__)

3

In [22]:
getsizeof({k:v for k,v in enumerate(range(3))})

240

A `normal` dict, with the same amount of elements, has the same size.

What if we add 10 new elements?

In [23]:
for i in range(10): my_instance_without_slots.__dict__[i] = str(i) 
getsizeof(my_instance_without_slots.__dict__), getsizeof(my_instance_without_slots)

(648, 56)

Let's go further with our dissertation about \__slots__ and compare them with a normal class in a dummy experiment.

In this example we import a json object (think about an api call) using both a normal class and a class with \__slots__

In [24]:
import json

my_json = '''{
    "username": "use@python3.org",
    "country": "Poland", "website":
    "www.chrisbarra.xzy",
    "date": "2017/08/15",
    "uid": 1, "gender": "Male"
}'''

In [25]:
class MyUserWithSlots():
    """A kind of user object"""
    
    __slots__ = ('username', 'country', 'website', 'date')
    
    def __init__ (self, username, country, website, date, **kwargs):
        self.username = username
        self.country = country
        self.website = website
        self.date = date

class MyUserWithoutSlots():
    """A kind of user object with slots"""
    
    def __init__ (self, username, country, website, date, **kwargs):
        self.username = username
        self.country = country
        self.website = website
        self.date = date
        
def get_size(instance):
    """
    If instance has __dict__ 
    we add the size of __dict__ 
    to the size of instance.
    
    In this way we correctly consider both
    size of the instance and of __dict__
    """
    size_dict = 0
    
    try:
        size_dict = getsizeof(instance.__dict__)
    except AttributeError:
        pass
    
    return size_dict + getsizeof(instance)

In [26]:
# we create 1.000.000 instances
NUM_INSTANCES = 1000000

In [27]:
# we create a list with the size of instance inside
with_slots = [get_size(MyUserWithSlots(**json.loads(my_json))) for _ in range(NUM_INSTANCES)]

# we sum the value inside the list
size_with_slots = sum(with_slots)/1000000

print(f"The total size is {size_with_slots} MB")

The total size is 72.0 MB


In [28]:
# we create a list with the size of instance inside
without_slots = [get_size(MyUserWithoutSlots(**json.loads(my_json))) for _ in range(NUM_INSTANCES)]

# we sum the value inside the list
size_without_slots = sum(without_slots)/1000000

print(f"The total size is {size_without_slots} MB")

The total size is 168.0 MB


In [29]:
size_reduction = ( size_with_slots - size_without_slots ) / size_without_slots * 100
print(f"Memory footprint reduction: {size_reduction:.2f}% ")

Memory footprint reduction: -57.14% 


## Wow! 
## ~57% less in memory usage with just one line of code. 

What about access time?

In [30]:
instance_with_slots = MyUserWithSlots(**json.loads(my_json))

In [31]:
%%timeit
z = instance_with_slots.username

58.2 ns ± 2.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [32]:
instance_without_slots = MyUserWithoutSlots(**json.loads(my_json))

In [33]:
%%timeit
z = instance_without_slots.username

72.4 ns ± 2.67 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


**\__slots__** are also slightly faster 👍

Want to know more about **\__slots__**?
Check the [official documentation](https://docs.python.org/3/reference/datamodel.html#object.__slots__)

## Questions for you

* What do you think about **\__slots__**? 
* Is there a use case where you have found **\__slots__** extremely useful?

This blog post is a notebook, you can download it from [here](index.ipynb)

Credits
* the picture is taken from [here](https://www.flickr.com/photos/geezaweezer/4753386960/in/photolist-kjEuwD-7dr4X-iBfQ7R-m9XzqT-k64Weg-ddpRxc-bostpX-4DLjZb-6zQjaR-pei5JY-gUnRct-mjTUpd-bsu9nZ-57FhEA-ejxXnc-qWSWoi-dX1DC1-bxHbpW-gUnSxA-rgaxET-kMGKSF-efXrc-jxuT4a-8mUREA-5aLrey-rUzhmu-gg5z-a8R2ZH-hSj2wt-fSEy3R-qDZLpQ-e6ABXa-ifXezw-6gvQGH-8HkDn-riKaDa-mjS5Y4-dpGQwC-dvEGV2-qedMiS-c5XXWU-kuHcwi-jnqsAc-h2KV8D-bdvsZe-buGSXF-8f3mkA-pRkbvg-pFsE33-3i3vw)